# 📘 Challenge TelecomX - Parte 2
## Modelos Preditivos para Churn

Este notebook dá continuidade ao projeto Challenge TelecomX (Parte 1), agora com foco na **modelagem preditiva da evasão de clientes (churn)**.

Seguiremos as etapas de pré-processamento, modelagem, avaliação e interpretação.

## 🔧 Importação de Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.inspection import permutation_importance
from imblearn.over_sampling import SMOTE
from sklearn.metrics import ConfusionMatrixDisplay

## 📥 Importação dos Dados Tratados

In [ ]:
df = pd.read_csv('dados_tratados.csv')
print(df.shape)
df.head()

## 🧹 Remoção de Colunas Irrelevantes

In [ ]:
if 'customerid' in df.columns:
    df.drop(columns=['customerid'], inplace=True)
    print("Coluna 'customerid' removida com sucesso!")
else:
    print("Nenhuma coluna 'customerid' encontrada.")

cols_variancia_zero = [col for col in df.columns if df[col].nunique() == 1]
if cols_variancia_zero:
    df.drop(columns=cols_variancia_zero, inplace=True)
    print(f"Colunas removidas por variância zero: {cols_variancia_zero}")
else:
    print("Nenhuma coluna com variância zero encontrada.")

## 🔡 Encoding de Variáveis Categóricas

In [ ]:
cat_cols = df.select_dtypes(include=['object']).columns.tolist()
print("Variáveis categóricas encontradas:", cat_cols)

if cat_cols:
    df = pd.get_dummies(df, columns=cat_cols, drop_first=True)
    print("One-Hot Encoding aplicado com sucesso!")
else:
    print("Não foram encontradas variáveis categóricas para codificação.")

## 📊 Verificação da Proporção de Evasão

In [ ]:
churn_counts = df['churn'].value_counts()
churn_percent = df['churn'].value_counts(normalize=True) * 100

print("Contagem de clientes por churn:")
print(churn_counts)
print("\nProporção (%):")
print(churn_percent.round(2))

plt.figure(figsize=(6,4))
churn_percent.plot(kind='bar', color=['skyblue','salmon'])
plt.title("Proporção de Clientes por Status de Evasão")
plt.xticks([0,1], labels=["Permaneceu (0)", "Evadido (1)"], rotation=0)
plt.ylabel("Percentual (%)")
plt.show()

## ⚖️ Balanceamento de Classes

In [ ]:
X = df.drop(columns=['churn'])
y = df['churn']

print("Distribuição original:")
print(y.value_counts())

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

print("\nDistribuição após SMOTE:")
print(y_resampled.value_counts())

## 📐 Normalização / Padronização

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_res_train, X_res_test, y_res_train, y_res_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42, stratify=y_resampled)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_res_train_scaled = scaler.fit_transform(X_res_train)
X_res_test_scaled = scaler.transform(X_res_test)

print("Pré-processamento concluído.")

## 🔗 Análise de Correlação

In [ ]:
corr_matrix = df.corr()
plt.figure(figsize=(14,10))
sns.heatmap(corr_matrix, annot=False, cmap='coolwarm', center=0)
plt.title("Matriz de Correlação - Variáveis Numéricas")
plt.show()

corr_target = corr_matrix['churn'].sort_values(ascending=False)
print("Correlação das variáveis com churn:")
print(corr_target)

top_corr = corr_target.abs().sort_values(ascending=False).head(11)
plt.figure(figsize=(8,6))
sns.barplot(x=top_corr.index, y=top_corr.values)
plt.title("Top 10 variáveis mais correlacionadas com churn")
plt.xticks(rotation=45)
plt.ylabel("Correlação (|valor|)")
plt.show()

## 🎯 Análises Direcionadas

In [ ]:
sns.boxplot(x='churn', y='tenure', data=df, palette='Set2')
plt.title("Tempo de Contrato vs Evasão")
plt.show()

sns.boxplot(x='churn', y='charges.total', data=df, palette='Set1')
plt.title("Total Gasto vs Evasão")
plt.show()

sns.scatterplot(x='tenure', y='charges.total', hue='churn', data=df, alpha=0.6, palette={0:'skyblue', 1:'salmon'})
plt.title("Tempo de Contrato vs Total Gasto (colorido por Evasão)")
plt.show()

## 🤖 Criação de Modelos

In [ ]:
log_reg = LogisticRegression(class_weight='balanced', random_state=42, max_iter=1000)
log_reg.fit(X_train_scaled, y_train)

rf = RandomForestClassifier(class_weight='balanced', random_state=42, n_estimators=200)
rf.fit(X_train, y_train)

print("Modelos treinados com sucesso!")

## 📊 Avaliação dos Modelos

In [ ]:
def avaliar_modelo(nome, modelo, X_train, X_test, y_train, y_test):
    print(f"\n=== Avaliação do Modelo: {nome} ===")
    y_pred_train = modelo.predict(X_train)
    y_pred_test = modelo.predict(X_test)
    acc_train = accuracy_score(y_train, y_pred_train)
    acc_test = accuracy_score(y_test, y_pred_test)
    print(f"Acurácia Treino: {acc_train:.4f}")
    print(f"Acurácia Teste: {acc_test:.4f}")
    print("\nRelatório de Classificação (Teste):")
    print(classification_report(y_test, y_pred_test))
    cm = confusion_matrix(y_test, y_pred_test)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=modelo.classes_)
    disp.plot(cmap="Blues")
    plt.title(f"Matriz de Confusão - {nome}")
    plt.show()
    return acc_train, acc_test

acc_lr_train, acc_lr_test = avaliar_modelo("Regressão Logística", log_reg, X_train_scaled, X_test_scaled, y_train, y_test)
acc_rf_train, acc_rf_test = avaliar_modelo("Random Forest", rf, X_train, X_test, y_train, y_test)

## 📋 Comparação de Métricas entre Modelos

In [ ]:
metricas = []
def obter_metricas(nome, modelo, X_test, y_test):
    y_pred = modelo.predict(X_test)
    return {"Modelo": nome, "Acurácia": accuracy_score(y_test, y_pred), "Precisão": precision_score(y_test, y_pred), "Recall": recall_score(y_test, y_pred), "F1-score": f1_score(y_test, y_pred)}

metricas.append(obter_metricas("Regressão Logística", log_reg, X_test_scaled, y_test))
metricas.append(obter_metricas("Random Forest", rf, X_test, y_test))
df_metricas = pd.DataFrame(metricas)
display(df_metricas)

df_plot = df_metricas.set_index("Modelo").T
df_plot.plot(kind="bar", figsize=(10,6))
plt.title("Comparação de Métricas entre Modelos")
plt.ylabel("Valor")
plt.ylim(0,1)
plt.xticks(rotation=0)
plt.show()

## 🌟 Importância das Variáveis

In [ ]:
importances = rf.feature_importances_
features = X.columns
feat_importances = pd.DataFrame({"Variável": features, "Importância": importances}).sort_values(by="Importância", ascending=False)
display(feat_importances.head(10))

sns.barplot(x="Importância", y="Variável", data=feat_importances.head(10), palette="viridis")
plt.title("Top 10 Variáveis mais Importantes - Random Forest")
plt.show()

# ✅ Conclusões Gerais e Recomendações

- Clientes com **contratos curtos** e **menor gasto total** apresentaram maior propensão à evasão.
- **Contratos mensais** e **pagamentos via cheque eletrônico** foram identificados como fatores de risco.
- **Random Forest** obteve melhor desempenho geral, mas com leve indício de overfitting.
- **Regressão Logística** apresentou resultados mais equilibrados e interpretáveis.

## 💡 Recomendações de Negócio
1. Focar em clientes novos com ações de retenção.
2. Incentivar contratos de longo prazo.
3. Criar ofertas especiais para clientes de baixo gasto.
4. Estimular métodos de pagamento automáticos.
5. Implementar monitoramento contínuo de churn.
